In [2]:
import csv
from bs4 import BeautifulSoup

In [3]:
from selenium import webdriver

In [4]:
driver = webdriver.Chrome()

In [5]:
url = 'https://www.lazada.com.ph/'
driver.get(url)

In [6]:
def get_url(search_term):
    """Generate url from search term"""
    template = 'https://www.lazada.com.ph/catalog/?_keyori=ss&from=input&page=1{}&spm=a2o4l.home.search.go.45d8359d4CpqYE'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [7]:
url = get_url('&q=laptop')
print(url)

https://www.lazada.com.ph/catalog/?_keyori=ss&from=input&page=1&q=laptop&spm=a2o4l.home.search.go.45d8359d4CpqYE


In [8]:
driver.get(url)

# Extract collection

In [9]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [10]:
results1 = soup.find_all('div', {'GridItem__info___3PztA'})

In [11]:
len(results1)

40

# Prototype the record

In [12]:
item = results1[0]

In [13]:
atag = item.a

In [14]:
description = atag.text

In [15]:
url = 'https://www.lazada.com.ph' + atag.get('href')

In [16]:
price = item.span

In [17]:
discountprice = price.text

# Generalize the pattern

In [31]:
def extract_record(item):
    """Extract and return data from a single record"""
    
    #description & url
    atag = item.a
    description = atag.text
    url = 'https:' + atag.get('href')
    
    #price
    price = item.span
    discountprice = price.text
    
    result = (description, url, discountprice)
    
    return result

In [32]:
records = []
results1 = soup.find_all('div', {'GridItem__info___3PztA'})

for item in results1:
    records.append(extract_record(item))

In [33]:
records[0]

('X201i core I3 processor portable thin office business learning laptop lowest price',
 'https://www.lazada.com.ph/products/x201i-core-i3-processor-portable-thin-office-business-learning-laptop-lowest-price-i1779486905-s7585830328.html?search=1',
 '₱7,307')

In [34]:
for row in records:
    print(row[1])

https://www.lazada.com.ph/products/x201i-core-i3-processor-portable-thin-office-business-learning-laptop-lowest-price-i1779486905-s7585830328.html?search=1
https://www.lazada.com.ph/products/asus-x515-x515ep-ej096t-156-gray-intel-core-i5-11th-gen-8gb-ram-mx330-free-asus-laptop-bag-pc-central-i2099225485-s9365383176.html?search=1
https://www.lazada.com.ph/products/laptop-sale-lenovo-x240-intel-i3-4th-gen-4gb-ram-500gb-hdd-intel-hd-grapics-wifi-ready-good-for-schooling-and-work-from-home-125-size-creen-i604914077-s1662710764.html?search=1
https://www.lazada.com.ph/products/laptop-lowest-price-salenec-intel-core-i7-2nd-gen-8gb-ram-120gb-ssd-intel-hd-graphics-12-inches-screen-free-webcam-gaming-laptop-i1486798835-s8665167165.html?search=1
https://www.lazada.com.ph/products/nec-versa-pro-va-e-notebook-laptop-intel-celeron-dual-core-4gb-ram-ddr3-250gb-hdd-free-laptop-bag-charger-we-also-have-desktop-pc-gaming-pc-laptop-desktop-package-pcsource-i1775102011-s7574662159.html?search=1
https://ww

# Next page

In [23]:
def get_url(search_term):
    """Generate url from search term"""
    template = 'https://www.lazada.com.ph/catalog/?_keyori=ss&from=input{}&spm=a2o4l.home.search.go.45d8359d4CpqYE'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url +='&page={}'
    
    return url

# Put all of it together

In [35]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
    """Generate url from search term"""
    template = 'https://www.lazada.com.ph/catalog/?_keyori=ss&from=input{}&spm=a2o4l.home.search.go.45d8359d4CpqYE'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    url +='&page={}'
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    
    #description & url
    atag = item.a
    description = atag.text
    url = 'https:' + atag.get('href')
    
    #price
    price = item.span
    discountprice = price.text
    
    result = (description, url, discountprice)
    
    return result

def main(search_term):
    """Run main program routine"""
    
    #startup the webdriver
    driver = webdriver.Chrome()
    
    record = []
    url = get_url(search_term)
    
    for page in range(1, 2):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results1 = soup.find_all('div', {'GridItem__info___3PztA'})
        
        for item in results1:
            record = extract_record(item)
            if record:
                records.append(record)
            
    driver.close()
    
    #save data to csv file
    with open('results1.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'URL', 'Price'])
        writer.writerows(records)

In [36]:
main('&q=laptop')